In [36]:
!git clone https://github.com/uclnlp/APE

Cloning into 'APE'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 47 (delta 19), reused 47 (delta 19), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [2]:
!git clone https://github.com/facebookresearch/FiD.git

Cloning into 'FiD'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 60 (delta 23), reused 39 (delta 14), pack-reused 3
Unpacking objects: 100% (60/60), done.


In [34]:
%%shell
bash FiD/get-data.sh

Saving data in open_domain_data
--2022-02-02 18:23:37--  https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4694541059 (4.4G) [application/gzip]
Saving to: ‘open_domain_data/download/psgs_w100.tsv.gz’

psgs_w100.tsv.gz    100%[===================>]   4.37G  42.5MB/s    in 1m 51s  

2022-02-02 18:25:29 (40.2 MB/s) - ‘open_domain_data/download/psgs_w100.tsv.gz’ saved [4694541059/4694541059]

Decompressing Wikipedia passages
--2022-02-02 18:30:07--  https://raw.githubusercontent.com/google-research-datasets/natural-questions/master/nq_open/NQ-open.dev.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.g

In [ ]:
import pandas as pd

in_csv = '/content/open_domain_data/psgs_w100.tsv'

number_lines = sum(1 for row in (open(in_csv)))

rowsize = round(number_lines / 8)

for i in range(1,number_lines,rowsize):
    df = pd.read_csv(in_csv, header=None, nrows = rowsize, skiprows = i , delimiter='\t')
    out_csv = 'input' + str(i) + '.csv'

    df.to_csv(out_csv,
          index=False,
          header=False,
          mode='a',#append data to csv file
          chunksize=rowsize)#size of data to append for each loop

In [37]:
%%shell

rm -r "open_domain_data/psgs_w100.tsv"

In [38]:
%%shell
ROOT="open_domain_data"
DOWNLOAD=$ROOT/"download"
echo "Processing ""$ROOT"
python FiD/src/preprocess.py $DOWNLOAD $ROOT

Processing open_domain_data


In [39]:
!pip install -r FiD/requirements.txt

     |████████████████████████████████| 8.6 MB 4.8 MB/s 
     |████████████████████████████████| 769 kB 47.9 MB/s 
     |████████████████████████████████| 1.2 MB 39.4 MB/s 
     |████████████████████████████████| 895 kB 46.8 MB/s 
     |████████████████████████████████| 3.0 MB 33.7 MB/s 


In [40]:
!pip install transformers==3.0.2
!pip install -r APE/requirements.txt

In [ ]:
%%shell

rm -r "open_domain_data/NQ"

In [ ]:
%%shell

rm -r "checkpoint"

In [ ]:
%%shell
python FiD/train_reader.py \
        --train_data open_domain_data/NQ/train.json \
        --eval_data open_domain_data/NQ/dev.json \
        --model_size base \
        --per_gpu_batch_size 1 \
        --n_context 1 \
        --name my_experiment \
        --checkpoint_dir checkpoint \
        --use_checkpoint 

[02/02/2022 13:43:53] {configuration_utils.py:264} INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json from cache at /root/.cache/torch/transformers/40578967d1f029acb6162b36db9d8b4307063e885990ccd297c2c5be1cf1b3d7.2995d650f5eba18c8baa4146e210d32d56165e90d374281741fc78b872cd6c9b
[02/02/2022 13:43:53] {configuration_utils.py:300} INFO - Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 

CalledProcessError: ignored

In [48]:
%%shell



python FiD/test_reader.py \
  --model_path pretrained_models/nq_reader_base \
  --eval_data open_domain_data/NQ/test.json \
  --model_size base \
  --per_gpu_batch_size 1 \
  --n_context 100 \
  --name my_test \
  --checkpoint_dir checkpoint


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
[02/02/2022 19:06:23] {configuration_utils.py:262} INFO - loading configuration file pretrained_models/nq_reader_base/config.json
[02/02/2022 19:06:23] {configuration_utils.py:300} INFO - Model config T5Config {
  "architectures": [
    "FiDT5"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_heads": 12,
  "num_layers": 1